<a href="https://colab.research.google.com/github/Gayatri1601R/LGMVIP-Data-Science/blob/main/LGM_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LGM-VIP Data Science Internship Programme**

Author - Gayatri Rajopadhye

**Advanced Level Task 2: Next Word Prediction**

**Importing libraries** 

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

**Loading Dataset**

In [ ]:
path = '1661-0.txt'
text = open(path).read().lower()
print('Corpus Length:', len(text))

Corpus Length: 581888


**Splitting Dataset into one word each**

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

**Making list of Sorted Unique Words**

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

**Feature Engineering**

In [ ]:
WORD_LENGTH = 4 
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the']
adventures


**Storing Feature and Corresponding Labels**

In [ ]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(X[0][0])

[False False False ... False False False]


**Building the model**

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

**Training our Model**

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/20
811/811 [==============================] - 258s 315ms/step - loss: 5.9067 - accuracy: 0.1140 - val_loss: 7.0806 - val_accuracy: 0.1001
Epoch 2/20
811/811 [==============================] - 245s 302ms/step - loss: 5.7662 - accuracy: 0.1503 - val_loss: 8.0600 - val_accuracy: 0.1075
Epoch 3/20
811/811 [==============================] - 225s 277ms/step - loss: 5.7184 - accuracy: 0.1785 - val_loss: 8.0752 - val_accuracy: 0.1104
Epoch 4/20
811/811 [==============================] - 225s 277ms/step - loss: 5.4269 - accuracy: 0.2135 - val_loss: 8.3298 - val_accuracy: 0.1022
Epoch 5/20
811/811 [==============================] - 222s 274ms/step - loss: 5.1517 - accuracy: 0.2523 - val_loss: 8.4610 - val_accuracy: 0.1009
Epoch 6/20
811/811 [==============================] - 221s 273ms/step - loss: 4.9039 - accuracy: 0.2909 - val_loss: 8.4502 - val_accuracy: 0.0868
Epoch 7/20
811/811 [==============================] - 221s 273ms/step - loss: 4.7166 - accuracy: 0.3281 - val_loss: 8.6499 -

**Loading Our Model**

In [13]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

**Making Predictions**

In [14]:
def prepare_input(text):
  x = np.zeros((1, WORD_LENGTH, len(unique_words)))
  for t, word in enumerate(text.split()):
    print(word)
    x[0, t, unique_word_index[word]] = 1
  return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [15]:
def sample(preds, top_n=3):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  return heapq.nlargest(top_n, range(len(preds)), preds.take)

**Function for Prediction**

In [17]:
def predict_completions(text, n=3):
  if text == "":
      return("0")
  x = prepare_input(text)
  preds = model.predict(x, verbose=0)[0]
  next_indices = sample(preds, n)
  return [unique_words[idx] for idx in next_indices]

In [18]:
s = "Will you be my partner?"
print("Sequence: ",s)
seq = " ".join(tokenizer.tokenize(s.lower())[0:2])
print("Sequence: ",seq)
print("Next Possible Words: ", predict_completions(seq, 2))

Sequence:  Will you be my partner?
Sequence:  will you
will
you
Next Possible Words:  ['really', 'nothing']


In [19]:
sen = "This is not going anywhere."
print("Sentence: ",sen)
sequ = " ".join(tokenizer.tokenize(sen.lower())[0:4])
print("Sequence: ",sequ)
print("Next Possible Words: ", predict_completions(sequ, 4))

Sentence:  This is not going anywhere.
Sequence:  this is not going
this
is
not
going
Next Possible Words:  ['on', 'up', 'as', 'out']
